In [1]:
import pandas as pd
import datetime

In [2]:
from models import arrival_history, util, config

In [3]:
pd.__version__

'0.24.1'

In [4]:
agency = config.get_agency('trimet')

In [5]:
agency_id = agency.id
route_id = '4'
version = 'v4c'
start_time_str = '00:00'
end_time_str = '24:00'

In [6]:
arrival_df = pd.DataFrame()
dates = util.get_dates_in_range('2022-03-10', '2022-03-10')
for date in dates:
    history = arrival_history.get_by_date(agency_id, route_id, date, version)
    arrival_data = history.get_data()
    route_id = arrival_data['route_id']
    agency = arrival_data['agency']
    data_start_time = arrival_data['start_time']
    data_end_time = arrival_data['end_time']
    stops_data_json = arrival_data['stops']
    if len(stops_data_json) > 0:
        for stop_id in stops_data_json.keys():
            for direction_id in stops_data_json[stop_id]['arrivals'].keys():
                one_stop_dir_arrivals = pd.DataFrame(stops_data_json[stop_id]['arrivals'][direction_id])
                one_stop_dir_arrivals['route_id'] = route_id
                one_stop_dir_arrivals['agency'] = agency
                one_stop_dir_arrivals['data_start_time'] = data_start_time
                one_stop_dir_arrivals['data_end_time'] = data_end_time
                one_stop_dir_arrivals['stop_id'] = stop_id
                one_stop_dir_arrivals['direction_id'] = direction_id
                one_stop_dir_arrivals['date'] = date
                if arrival_df.empty:
                    arrival_df = one_stop_dir_arrivals.copy()
                else:
                    arrival_df = arrival_df.append(one_stop_dir_arrivals)
    else:
        print(f"No arrivals for route_id: {route_id} date: {date}")
        arrivals_df = pd.DataFrame([f"No arrivals for route_id: {route_id} date: {date}"], columns=['error_message'])

In [7]:
rename_dict = {'t':'time','e':'departure_time','d':'distance','i':'trip_id', 'v':'vehicle_id'}
arrival_w_names_df = arrival_df.rename(columns=rename_dict).copy()

In [10]:
arrival_w_names_df.head()

,distance,departure_time,trip_id,time,vehicle_id,route_id,agency,data_start_time,data_end_time,stop_id,direction_id,date
0,16,1646915743,511,1646915710,4232,4,trimet,1646910000,1646996400,10177,1,2022-03-10
1,15,1646919391,60,1646919353,3004,4,trimet,1646910000,1646996400,10177,1,2022-03-10
2,14,1646920196,326,1646920192,3640,4,trimet,1646910000,1646996400,10177,1,2022-03-10
3,15,1646921248,145,1646921233,3231,4,trimet,1646910000,1646996400,10177,1,2022-03-10
4,10,1646921972,434,1646921942,4041,4,trimet,1646910000,1646996400,10177,1,2022-03-10


In [12]:
arrival_w_names_df.sort_values(['trip_id', 'time']).head(3)

,distance,departure_time,trip_id,time,vehicle_id,route_id,agency,data_start_time,data_end_time,stop_id,direction_id,date
3,56,1646925111,0,1646925110,2801,4,trimet,1646910000,1646996400,6387,0,2022-03-10
3,243,1646925112,0,1646925112,2801,4,trimet,1646910000,1646996400,10611,0,2022-03-10
3,50,1646925113,0,1646925113,2801,4,trimet,1646910000,1646996400,1045,0,2022-03-10
